In [1]:
import logging

from automata.cli.commands import reconfigure_logging
from automata.config.base import AgentConfigName
from automata.config.openai_agent import OpenAIAutomataAgentConfigBuilder
from automata.agent.providers import OpenAIAutomataAgent
from automata.singletons.dependency_factory import dependency_factory
from automata.singletons.py_module_loader import py_module_loader
from automata.tools.factory import AgentToolFactory

logger = logging.getLogger(__name__)
reconfigure_logging("DEBUG")

py_module_loader.initialize()

In [2]:
# Construct the set of all dependencies that will be used to build the tools
toolkit_list = ["context-oracle"]
tool_dependencies = dependency_factory.build_dependencies_for_tools(toolkit_list)

INFO:automata.tools.factory:Building dependencies for toolkit_list ['context-oracle']...
INFO:automata.tools.factory:Building symbol_code_embedding_handler...
INFO:automata.tools.factory:Creating dependency symbol_code_embedding_handler
INFO:automata.tools.factory:Building symbol_search...
INFO:automata.tools.factory:Creating dependency symbol_search
INFO:automata.tools.factory:Creating dependency symbol_graph
INFO:automata.tools.factory:Creating dependency embedding_similarity_calculator
INFO:automata.tools.factory:Building embedding_similarity_calculator...
INFO:automata.tools.factory:Building symbol_doc_embedding_handler...
INFO:automata.tools.factory:Creating dependency symbol_doc_embedding_handler
INFO:automata.tools.factory:Creating dependency py_context_retriever


In [3]:
# Build the tools
tools = AgentToolFactory.build_tools(toolkit_list, **tool_dependencies)

In [7]:
# Build the agent config
config_name = AgentConfigName("automata-main")

agent_config = (
    OpenAIAutomataAgentConfigBuilder.from_name(config_name)
    .with_tools(tools)
    .with_model("gpt-4")
    .with_max_iterations(2)
    .build()
)

In [8]:
# Initialize the agent
# instructions = "Explain how OpenAI is used by the codebase. Note in particularly how it is used in embeddings and how agents consume it"
instructions = "Provide a comprehensive explanation of how SymbolRank and SymbolSearch work. Return your answer as a valid JSON string."
agent = OpenAIAutomataAgent(instructions, config=agent_config)

DEBUG:automata.agent.providers:Setting up agent with tools = [OpenAITool(function=<bound method ContextOracleToolkitBuilder._get_context of <automata.tools.builders.context_oracle.ContextOracleOpenAIToolkitBuilder object at 0x120d960d0>>, name='context-oracle', description="This tool utilizes the EmbeddingSimilarityCalculator and SymbolSearch to provide context for a given query by computing semantic similarity between the query and all available symbols' documentation and code. The symbol with the highest combined similarity score is identified, with its source code and documentation summary forming the primary context. Additionally, if enabled, the documentation summaries of related symbols (those next most similar to the query) are included.", coroutine=None, properties={'query': {'type': 'string', 'description': 'The query string to search for.'}, 'max_additional_related_symbols': {'type': 'integer', 'description': 'The maximum number of additional related symbols to return documen

DEBUG:automata.agent.providers:
------------------------------------------------------------
Session ID: 5453553f-a7d3-42d6-a663-1f966aa7604a
------------------------------------------------------------




In [9]:
# Run the agent
result = agent.run()

DEBUG:root:
------------------------------------------------------------------------------------------------------------------------
Latest Assistant Message -- 

DEBUG:automata.llm.providers.openai:Approximately 2415 tokens were consumed prior to completion generation.


Function Call:
context-oracle

Arguments:
{
  "query": "SymbolRank"
}

 

DEBUG:automata.llm.providers.openai:Approximately 2450 tokens were after adding the latest message.
DEBUG:root:
------------------------------------------------------------------------------------------------------------------------
DEBUG:automata.agent.providers:Latest User Message -- 
user:
content=Execution Result:

@dataclass
class Symbol:
    """
    A class which contains associated logic for a Symbol.

    A Symbol specifies a python class, method, or a local variable.
    A Symbol has a standardized string representation called a URI. The syntax for Symbol is the following:

    # (<x>)+ stands for one or more repetitions of <x>
    <symbol>               ::= <scheme> ' ' <package> ' ' (<descriptor>)+ | 'local ' <local-id>
    <package>              ::= <manager> ' ' <package-name> ' ' <version>
    <scheme>               ::= any UTF-8, escape spaces with double space.
    <manager>              ::= same as above, use the placeholder '.' to indicate an empty value
    <package-

DEBUG:automata.llm.providers.openai:Approximately 6336 tokens were after adding the latest message.
DEBUG:root:
------------------------------------------------------------------------------------------------------------------------
DEBUG:root:
------------------------------------------------------------------------------------------------------------------------
Latest Assistant Message -- 

DEBUG:automata.llm.providers.openai:Approximately 6336 tokens were consumed prior to completion generation.


Function Call:
call_termination

Arguments:
{
  "result": "{\"SymbolRank\": {\"description\": \"SymbolRank is a class that uses the PageRank algorithm to rank symbols within a software's semantic and structural context. It works by constructing a SymbolGraph, wherein every symbol, such as a class or method, in a codebase is a node and dependencies form edges. This graph, along with a similarity dictionary, assists in computing SymbolRanks, which denote each symbol's prominence within the software. This process, similar to how Google PageRank operates, requires a SymbolRankConfig to provide necessary computation parameters.\", \"methods\": {\"get_ranks\": \"Retrieve the SymbolRanks of each symbol\"}}, \"SymbolSearch\": {\"description\": \"The SymbolSearch class facilitates the seamless search of symbols within an indexed codebase using various related symbols and methods such as SymbolGraph, SymbolRankConfig, SymbolEmbeddingHandler, and EmbeddingSimilarityCalculator. It is equipped with

DEBUG:automata.llm.providers.openai:Approximately 6700 tokens were after adding the latest message.
DEBUG:root:
------------------------------------------------------------------------------------------------------------------------
DEBUG:automata.agent.providers:Latest User Message -- 
user:
content=Execution Result:

{"SymbolRank": {"description": "SymbolRank is a class that uses the PageRank algorithm to rank symbols within a software's semantic and structural context. It works by constructing a SymbolGraph, wherein every symbol, such as a class or method, in a codebase is a node and dependencies form edges. This graph, along with a similarity dictionary, assists in computing SymbolRanks, which denote each symbol's prominence within the software. This process, similar to how Google PageRank operates, requires a SymbolRankConfig to provide necessary computation parameters.", "methods": {"get_ranks": "Retrieve the SymbolRanks of each symbol"}}, "SymbolSearch": {"description": "The Sym

In [10]:
# Print the result
print(f"Result:\n{result}")

Result:
Execution Result:

{"SymbolRank": {"description": "SymbolRank is a class that uses the PageRank algorithm to rank symbols within a software's semantic and structural context. It works by constructing a SymbolGraph, wherein every symbol, such as a class or method, in a codebase is a node and dependencies form edges. This graph, along with a similarity dictionary, assists in computing SymbolRanks, which denote each symbol's prominence within the software. This process, similar to how Google PageRank operates, requires a SymbolRankConfig to provide necessary computation parameters.", "methods": {"get_ranks": "Retrieve the SymbolRanks of each symbol"}}, "SymbolSearch": {"description": "The SymbolSearch class facilitates the seamless search of symbols within an indexed codebase using various related symbols and methods such as SymbolGraph, SymbolRankConfig, SymbolEmbeddingHandler, and EmbeddingSimilarityCalculator. It is equipped with key functions like 'exact_search' to locate exac

In [11]:
result

'Execution Result:\n\n{"SymbolRank": {"description": "SymbolRank is a class that uses the PageRank algorithm to rank symbols within a software\'s semantic and structural context. It works by constructing a SymbolGraph, wherein every symbol, such as a class or method, in a codebase is a node and dependencies form edges. This graph, along with a similarity dictionary, assists in computing SymbolRanks, which denote each symbol\'s prominence within the software. This process, similar to how Google PageRank operates, requires a SymbolRankConfig to provide necessary computation parameters.", "methods": {"get_ranks": "Retrieve the SymbolRanks of each symbol"}}, "SymbolSearch": {"description": "The SymbolSearch class facilitates the seamless search of symbols within an indexed codebase using various related symbols and methods such as SymbolGraph, SymbolRankConfig, SymbolEmbeddingHandler, and EmbeddingSimilarityCalculator. It is equipped with key functions like \'exact_search\' to locate exact